**\# Database Schema Diagram**

\<img src="file:///C:/Users/Jayvy\_nei0kss/Downloads/DDL.drawio" alt="Database Schema Diagram" width="800"/\>

**\# Data Mart Creation and Data Loading**

\## DDL (Table Builds)

\### Schema Creation

**Explanation: Creating the required schemas for dimensions and facts.**

<mark>\-- SQL code for creating schemas</mark>

IF NOT EXISTS (SELECT \* FROM sys.schemas WHERE name = 'dim' ) 

BEGIN

    EXEC sp\_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'

END

;

IF NOT EXISTS (SELECT \* FROM sys.schemas WHERE name = 'stg' ) 

BEGIN

    EXEC sp\_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'

END

;

IF NOT EXISTS (SELECT \* FROM sys.schemas WHERE name = 'f' ) 

BEGIN

    EXEC sp\_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'

END

;

\## Customer Dimension Table

  

\*\*Explanation:\*\*

Creating the Customer dimension table.

  

\*\*SQL Code:\*\*

\`\`\`sql

\-- SQL code for creating dim.Customers table

IF NOT EXISTS (SELECT 1 FROM INFORMATION\_SCHEMA.TABLES WHERE TABLE\_SCHEMA = 'dim' AND TABLE\_NAME = 'Customers')

BEGIN

    CREATE TABLE dim.Customers(

        pkCustomer int IDENTITY(1000,1) NOT NULL,

        CustomerID nvarchar(5) NOT NULL,

        Customer nvarchar(40) NOT NULL,

        City nvarchar(15) NULL,

        Country nvarchar(15) NULL,

        LoadDate DATE NOT NULL,

        SourceCountry nvarchar(25) NULL

    );

  

    -- Add constraints

    ALTER TABLE dim.Customers

    ADD CONSTRAINT PK\_Customers\_LUP PRIMARY KEY(pkCustomer);

  

    ALTER TABLE dim.Customers

    ADD CONSTRAINT UC\_Customers\_ID UNIQUE (CustomerID);

END

**Calendar Dimension Table**  
<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space-collapse: preserve;"><b>Explanation: Creating the Calendar dimension table.</b></span>  
<mark>\-- SQL code for creating dim.Calendar table</mark>

IF NOT EXISTS (SELECT \* FROM INFORMATION\_SCHEMA.TABLES WHERE TABLE\_SCHEMA = 'dim' AND TABLE\_NAME = 'Calendar')

BEGIN

    CREATE TABLE dim.Calendar

    (

        pkCalendar INT NOT NULL,

        DateValue DATE NOT NULL,

        Year INT NOT NULL,

        Quarter INT NOT NULL,

        Qtr VARCHAR(3) NOT NULL,

        Month INT NOT NULL,

        MonthName VARCHAR(10) NOT NULL,

        MonthShort VARCHAR(3) NOT NULL,

        Week INT NOT NULL,

        Day INT NOT NULL,

        DayName VARCHAR(10) NOT NULL,

        DayShort VARCHAR(3) NOT NULL,

        IsWeekday BIT NOT NULL,

        Weekday VARCHAR(3) NOT NULL

    );

    -- Add constraints

    ALTER TABLE dim.Calendar

    ADD CONSTRAINT PK\_Calendar\_Julian PRIMARY KEY(pkCalendar);

    ALTER TABLE dim.Calendar

    ADD CONSTRAINT UC\_Calendar UNIQUE (DateValue);

END

**Shipper Dimension Table  
  
Explanation:- Creating the shipper dimension table.**

  

<mark>\-- SQL code for creating dim.Shipper table</mark>

IF NOT EXISTS (SELECT \* FROM INFORMATION\_SCHEMA.TABLES WHERE TABLE\_SCHEMA = 'dim' AND TABLE\_NAME = 'Shipper')

BEGIN

    CREATE TABLE dim.Shipper

    (

        pkShipId int not null,

        Shipper nvarchar(40) not null,

        Phone nvarchar(24) null

    );

    -- Add constraints

    ALTER TABLE dim.Shipper

    ADD CONSTRAINT PK\_Ship PRIMARY KEY(pkShipId);

END

**OrderPerf Fact Table**  
  
**Explanation- Creating the OrderPerf fact table.**  
  

<mark>\-- SQL code for creating f.OrderPerf table</mark>

IF EXISTS (SELECT \* FROM INFORMATION\_SCHEMA.TABLES WHERE TABLE\_SCHEMA = 'f' AND TABLE\_NAME = 'OrderPerf')

BEGIN 

    DROP TABLE f.OrderPerf;

END

  

CREATE TABLE f.OrderPerf(

    OrderID int NOT NULL,

    fkCalendar int NOT NULL,

    fkCustomer int NOT NULL,

    fkShipper int NULL,

    fkEmployee int NULL,

    Freight money NOT NULL,

    DaysToShip int NULL,

    DaysTilRequired int NULL,

    OrderCount int NOT NULL

);

  

\-- Add constraints

ALTER TABLE f.OrderPerf

ADD CONSTRAINT PK\_ORD PRIMARY KEY(OrderID)

;

  

ALTER TABLE f.OrderPerf

ADD CONSTRAINT FK\_ORDtoCAL

    FOREIGN KEY (fkCalendar)              

    REFERENCES  dim.Calendar(pkCalendar) 

;

  

ALTER TABLE f.OrderPerf

ADD CONSTRAINT FK\_ORDtoCUST

    FOREIGN KEY (fkCustomer)              

    REFERENCES  dim.Customers(pkCustomer) 

;

  

ALTER TABLE f.OrderPerf

ADD CONSTRAINT FK\_ORDtoSHIP

    FOREIGN KEY (fkShipper)

    REFERENCES dim.Shipper(pkShipId)

;

  

ALTER TABLE f.OrderPerf

ADD CONSTRAINT FK\_ORDtoEMP

    FOREIGN KEY (fkEmployee)

    REFERENCES dim.Employees(EmployeeID)

;

# **DATA LOADING SCRIPT**

**Loading Data Into dim.Customers**

**Explanation- Loading data into the customer dimension table.**

<mark>\-- SQL code for loading data into dim.Customers</mark>

INSERT INTO dim.Customers (CustomerID, Customer, City, Country, LoadDate, SourceCountry)

    SELECT nwc.CustomerID, nwc.CompanyName, nwc.City, nwc.Country, CAST(GETDATE() AS DATE), CONCAT('Country = ', nwc.Country)

    FROM Northwind\_2023.dbo.Customers nwc

    WHERE nwc.CustomerID NOT IN (SELECT CustomerID FROM dim.Customers);

**Loading Data into dim.Calendar**  
**Explanation- Loading data into the Customer dimension table.**  
  

<mark>\-- SQL code for loading data into dim.Calendar</mark>

DECLARE @StartDate DATE = '2020-01-01'

DECLARE @EndDate DATE = DATEADD(YEAR, 0, GETDATE())

DECLARE @Date DATE = @StartDate

DECLARE @DayID INT = (DATEPART(YEAR, @StartDate) - 1900) \* 1000 + DATEPART(DY, @StartDate);

  

WHILE @Date \<= @EndDate

BEGIN

    INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Quarter, Qtr, Month, MonthName, MonthShort, Week, Day, DayName, DayShort, IsWeekday, Weekday)

    VALUES (

        @DayID,

        @Date,

        YEAR(@Date),

        DATEPART(QUARTER, @Date),

        CASE WHEN DATEPART(QUARTER, @Date) IN (1) THEN '1st'

            WHEN DATEPART(QUARTER, @Date) IN (2) THEN '2nd'

            WHEN DATEPART(QUARTER, @Date) IN (3) THEN '3rd'

            WHEN DATEPART(QUARTER, @Date) IN (4) THEN '4th'

            ELSE '5th'

        END,

        MONTH(@Date),

        DATENAME(MONTH, @Date),

        LEFT(DATENAME(MONTH, @Date), 3),

        DATEPART(WEEK, @Date),

        DAY(@Date),

        DATENAME(WEEKDAY, @Date),

        LEFT(DATENAME(WEEKDAY, @Date), 3),

        CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 0 ELSE 1 END,

        CASE WHEN